In [1]:
import datetime

from util.util import importstr
from util.logconf import logging
log = logging.getLogger('nb')

In [2]:
# eg: app='p2ch11.training.NoduleClsTrainingApp'
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # <1>
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))  # <2> #app_cls=NoduleClsTrainingApp()
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}).main()".format(app, argv))

In [3]:
import os
import shutil

# clean up any old data that might be around.
# We don't call this by default because it's destructive, 
# and would waste a lot of time if it ran when nothing 
# on the application side had changed.
def cleanCache():
    shutil.rmtree('data/cache')
    os.mkdir('data/cache')

# cleanCache()

In [5]:
training_epochs = 20
experiment_epochs = 5
final_epochs = 50

training_epochs = 2
experiment_epochs = 5
final_epochs = 5
seg_epochs = 10

### Run Nodule Classification Model

In [ ]:
run('nodule_classification_model.prepcache.LunaPrepCacheApp')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', '--epochs=1', 'unbalanced')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={5}', '--balanced', 'balanced')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={5}', '--balanced', '--augment-flip', 'flip')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-offset', 'offset')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-scale', 'scale')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={experiment_epochs}', '--balanced', '--augment-rotate', 'rotate')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={15}', '--balanced', '--augment-noise', 'noise')

In [ ]:
run('nodule_classification_model.training.NoduleClsTrainingApp', f'--epochs={15}', '--balanced', '--augmented', 'fully-augmented')

### Run segmentation model

In [ ]:
run('segmentation_model.prepcache.LunaPrepCacheApp')

In [ ]:
run('segmentation_model.training.SegmentationTrainingApp', f'--epochs={20}', '--balanced', '--augmented', 'final-cls')

In [ ]:
run('segmentation_model.training.NoduleClsTrainingApp', f'--epochs={20}', '--balanced', '--augment-flip', 'final-cls')

In [ ]:
run('segmentation_model.training.SegmentationTrainingApp', f'--epochs={20}', '--augmented', 'final-cls')

### Run Malignancy classification

In [ ]:
run('malignancy_classification_model.prepcache.LunaPrepCacheApp')

In [ ]:
run('malignancy_classification_model.training.malignancyClassificationTrainingApp', f'--epochs=10', 'nodule-nonnodule')

In [ ]:
run('malignancy_classification_model.training.malignancyClassificationTrainingApp', f'--epochs=40', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=''data/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state',
    'finetune-head')


In [ ]:
run('malignancy_classification_model.training.malignancyClassificationTrainingApp', f'--epochs=40', '--malignant', '--dataset=MalignantLunaDataset',
    '--finetune=''data/models/cls_2020-02-06_14.16.55_final-nodule-nonnodule.best.state',
    '--finetune-depth=2',
    'finetune-depth2')


### End-to-end program

In [4]:
run('nodule_analysis.NoduleAnalysisApp', '--run-validation')

2025-03-10 21:35:23,150 INFO     pid:11436 nb:005:run Running: nodule_analysis.NoduleAnalysisApp(['--num-workers=4', '--run-validation']).main()
2025-03-10 21:35:27,876 DEBUG    pid:11436 nodule_analysis:219:initModels data/models/seg_2020-01-26_19.45.12_w4d3c1-bal_1_nodupe-label_pos-d1_fn8-adam.best.state
d:\GitHub folders\Lung_cancer_Detector\nodule_analysis.py:220: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `tor

1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1028 |                3
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0
Total
                 |    Complete Miss |     Filtered Out |     Pred. Nodule
     Non-Nodules |                  |             1028 |                3
          Benign |                0 |                0 |                0
       Malignant |                0 |                0 |                0


In [ ]:
run('nodule_analysis.NoduleAnalysisApp', '--run-validation', '--malignancy-path')

In [ ]:
run('nodule_analysis.NoduleAnalysisApp', 'series_uid=1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866')
#something is wrong here